In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import DistanceMetric  
from datetime import datetime, timedelta

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#Deal with SettingCopyWarnings
pd.options.mode.chained_assignment = None  # default='warn'

## Step 1: Create a dataset of unique fire- nearest weather station pairs

### Preparing a dataframe containing unique weather station coordinates

In [4]:
#Reading the file which has coordinates of all the weather stations across the 15 "climate daily" csv files
ctc1 = pd.read_csv("climatetotalcoord1.csv")

In [5]:
#limiting decimal digits to 4
ctc1 = round(ctc1, 4)

In [6]:
#We are creating a new column which merges latitude and longitude to form a new combined column called coordinates. 
#But since the "+" operator only works on strings, we are also converting the lat and long into  strings using astype(). The "," acts as a separator. Beware of NaNs when doing this
ctc1["coordinates"] = ctc1["latitude"].astype(str) + "," + ctc1["longitude"].astype(str)

In [7]:
#Finding the unique values in the coordinate column
ctc2 = ctc1["coordinates"].unique()

In [8]:
ctc3 = pd.DataFrame(ctc2, columns = ["climcoordinates"])

In [9]:
#Splitting the "coordinates1" column to two separate columns (climlatitude & climlongitude) on the basis of the comma delimitor
ctc3[["climlatitude", "climlongitude"]] = ctc3.climcoordinates.str.split(",", expand = True)

In [10]:
#Converting climlatitude and climlongitude into float
ctc3["climlatitude"] = ctc3["climlatitude"].astype(float)
ctc3["climlongitude"] = ctc3["climlongitude"].astype(float)

### Preparing a wildfire dataset

In [11]:
wild2 = pd.read_csv("canadawildfiresupdated1_2011to2021.csv")

In [12]:
#We are creating a new column which merges latitude and longitude to form a new combined column called coordinates. 
wild2["coordinates"] = wild2["latitude"].astype(str) + "," + wild2["longitude"].astype(str)

In [13]:
#Limiting decimal points to 4
wild2 = round(wild2, 4)

In [14]:
#Codename Quebec
#Making a dataframe for only the Quebec region
mask = wild2["src_agency"] == "QC"
fire2_qc = wild2[mask]

In [20]:
fire2_qc.head(2)

,Unnamed: 0,fid,src_agency,latitude,longitude,date,sizeha,cause,protzone,ecoz_name,wildlat1,coordinates
47903,47903,328420,QC,52.5747,-76.5213,01-07-2019,408.5,L,nordique,Hudson Plain,"(50, 55]","52.5747,-76.5213"
47904,47904,328421,QC,52.5730,-76.5418,01-07-2019,34.5,L,nordique,Hudson Plain,"(50, 55]","52.573,-76.5418"


In [21]:
ctc3.head(2)

,climcoordinates,climlatitude,climlongitude
0,"68.2233,-135.0058",68.2233,-135.0058
1,"68.2233,-135.0056",68.2233,-135.0056


### Calculating and adding a column of haversine distance

In [15]:
#Crossjoining quebec dataset
cj5 = fire2_qc.merge(ctc3, how = "cross")

In [16]:
#Converting fire and weather(climate) station data of quebec to radians for applying haversine formula
cj5[["firelat_radians","firelong_radians"]] = np.radians(cj5.loc[:,["latitude", "longitude"]])
cj5[["climlat_radians", "climlong_radians"]] = np.radians(cj5.loc[:,["climlatitude", "climlongitude"]])

In [17]:
#Defining the haversine formula
def haversine_distance(lon1, lat1, lon2, lat2):
    newlat = lat2 - lat1
    newlon = lon2 - lon1

    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2

    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    # use appropriate value for radius of the earth (this is crude!)
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km

In [18]:
#To compute distances by applying the formula
cj5["distance_km"] = haversine_distance(cj5["firelong_radians"], cj5["firelat_radians"], cj5["climlong_radians"], 
                                                  cj5["climlat_radians"])

### Creating the dataset of unique fire - nearest weather station pairs

In [19]:
#We group by the entire dataset on the basis of fire identity (fid) and then select the rows with 
#minimum value for distance from weather station
cj6qc = cj5.groupby(["fid"])["distance_km"].min()

In [20]:
#Converting cj6qc, which is a series object, into a dataframe
cj6qc = pd.DataFrame(cj6qc, columns = ["distance_km"])

In [21]:
#Now make index of cj6qc as column "fid" and change index
cj6qc['fid'] = cj6qc.index
cj6qc.index = range(cj6qc.shape[0])

In [22]:
#Now merging cj5 and cj6qc on both columns "fid" and "distance_km"
cj7qc = pd.merge(cj5, cj6qc, on = ["fid", "distance_km"])

In [31]:
cj7qc.head(2)

,Unnamed: 0,fid,src_agency,latitude,longitude,date,sizeha,cause,protzone,ecoz_name,wildlat1,coordinates,climcoordinates,climlatitude,climlongitude,firelat_radians,firelong_radians,climlat_radians,climlong_radians,distance_km
0,47903,328420,QC,52.5747,-76.5213,01-07-2019,408.5,L,nordique,Hudson Plain,"(50, 55]","52.5747,-76.5213","53.6253,-77.7042",53.6253,-77.7042,0.917602,-1.335549,0.935938,-1.356194,140.918574
1,47904,328421,QC,52.5730,-76.5418,01-07-2019,34.5,L,nordique,Hudson Plain,"(50, 55]","52.573,-76.5418","52.2264,-78.5225",52.2264,-78.5225,0.917572,-1.335906,0.911523,-1.370476,139.705991


## Step 2: Create a dataset containing each fire + climate pair data and climate data from 15 days prior

### Creating a mega climate/weather station dataset

In [39]:
#Reading all the climate data files into one sheet
cd1 = pd.read_csv("climate-daily-1.csv")
cd2 = pd.read_csv("climate-daily-2.csv")
cd3 = pd.read_csv("climate-daily-3.csv")
cd4 = pd.read_csv("climate-daily-4.csv")
cd5 = pd.read_csv("climate-daily-5.csv")
cd6 = pd.read_csv("climate-daily-6.csv")
cd7_1 = pd.read_csv("climate-daily-7.1.csv")
cd7_2 = pd.read_csv("climate-daily-7.2.csv")
cd8 = pd.read_csv("climate-daily-8.csv")
cd9 = pd.read_csv("climate-daily-9.csv")
cd10 = pd.read_csv("climate-daily-10.csv")
cd11 = pd.read_csv("climate-daily-11.csv")
cd12 = pd.read_csv("climate-daily-12.csv")
cd13 = pd.read_csv("climate-daily-13.csv")
cd14 = pd.read_csv("climate-daily-14.csv")

C:\Users\ROHAN\AppData\Local\Temp\ipykernel_9380\1498047778.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  cd2 = pd.read_csv("climate-daily-2.csv")
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_9380\1498047778.py:4: DtypeWarning: Columns (23,28,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  cd3 = pd.read_csv("climate-daily-3.csv")
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_9380\1498047778.py:5: DtypeWarning: Columns (4,25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  cd4 = pd.read_csv("climate-daily-4.csv")
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_9380\1498047778.py:7: DtypeWarning: Columns (12,15,18,19,23) have mixed types. Specify dtype option on import or set low_memory=False.
  cd6 = pd.read_csv("climate-daily-6.csv")
C:\Users\ROHAN\AppData\Local\Temp\ipykernel_9380\1498047778.py:8: DtypeWarning: Columns (7,12,21,28,31) have mixed types. Specify dtyp

In [40]:
#Convert all column names into lowercase
cd1.columns = cd1.columns.str.lower()
cd2.columns = cd2.columns.str.lower()
cd3.columns = cd3.columns.str.lower()
cd4.columns = cd4.columns.str.lower()
cd5.columns = cd5.columns.str.lower()
cd6.columns = cd6.columns.str.lower()
cd7_1.columns = cd7_1.columns.str.lower()
cd7_2.columns = cd7_2.columns.str.lower()
cd8.columns = cd8.columns.str.lower()
cd9.columns = cd9.columns.str.lower()
cd10.columns = cd10.columns.str.lower()
cd11.columns = cd11.columns.str.lower()
cd12.columns = cd12.columns.str.lower()
cd13.columns = cd13.columns.str.lower()
cd14.columns = cd14.columns.str.lower()

In [41]:
#Rearranging all columns alphabetically
cd1 = cd1.reindex(sorted(cd1.columns), axis = 1)
cd2 = cd2.reindex(sorted(cd2.columns), axis = 1)
cd3 = cd3.reindex(sorted(cd3.columns), axis = 1)
cd4 = cd4.reindex(sorted(cd4.columns), axis = 1)
cd5 = cd5.reindex(sorted(cd5.columns), axis = 1)
cd6 = cd6.reindex(sorted(cd6.columns), axis = 1)
cd7_1 = cd7_1.reindex(sorted(cd7_1.columns), axis = 1)
cd7_2 = cd7_2.reindex(sorted(cd7_2.columns), axis = 1)
cd8 = cd8.reindex(sorted(cd8.columns), axis = 1)
cd9 = cd9.reindex(sorted(cd9.columns), axis = 1)
cd10 = cd10.reindex(sorted(cd10.columns), axis = 1)
cd11 = cd11.reindex(sorted(cd11.columns), axis = 1)
cd12 = cd12.reindex(sorted(cd12.columns), axis = 1)
cd13 = cd13.reindex(sorted(cd13.columns), axis = 1)
cd14 = cd14.reindex(sorted(cd14.columns), axis = 1)

In [42]:
#Concatenating all datasets to create one mega climate dataset
cd1_1 = pd.concat([cd1,cd2])
cd1_2 = pd.concat([cd1_1, cd3])
cd1_3 = pd.concat([cd1_2, cd4])
cd1_4 = pd.concat([cd1_3, cd5])
cd1_5 = pd.concat([cd1_4, cd6])
cd1_6 = pd.concat([cd1_5, cd7_1])
cd1_7 = pd.concat([cd1_6, cd7_2])
cd1_8 = pd.concat([cd1_7, cd8])
cd1_9 = pd.concat([cd1_8, cd9])
cd1_10 = pd.concat([cd1_9, cd10])
cd1_11 = pd.concat([cd1_10, cd11])
cd1_12 = pd.concat([cd1_11, cd12])
cd1_13 = pd.concat([cd1_12, cd13])
cdtotal = pd.concat([cd1_13, cd14])

In [43]:
cdtotal = round(cdtotal, 4)

In [79]:
cdtotal.head(2)

,climate_identifier,cooling_degree_days,cooling_degree_days_flag,direction_max_gust,direction_max_gust_flag,heating_degree_days,heating_degree_days_flag,id,local_date,local_day,local_month,local_year,max_rel_humidity,max_rel_humidity_flag,max_temperature,max_temperature_flag,mean_temperature,mean_temperature_flag,min_rel_humidity,min_rel_humidity_flag,min_temperature,min_temperature_flag,province_code,snow_on_ground,snow_on_ground_flag,speed_max_gust,speed_max_gust_flag,station_name,total_precipitation,total_precipitation_flag,total_rain,total_rain_flag,total_snow,total_snow_flag,x,y,climdate_formatted
0,2200100,1.2,NaN,NaN,NaN,0.0,NaN,2200100.2012.8.2,2012-08-02,2,8,2012,NaN,NaN,22.3,NaN,19.2,NaN,NaN,NaN,16.0,NaN,NT,0.0,NaN,NaN,NaN,AKLAVIK A,0.0,T,0.0,T,0.0,NaN,-135.0058,68.2233,02/08/2012
1,2200100,0.0,NaN,NaN,NaN,1.8,NaN,2200100.2012.8.4,2012-08-04,4,8,2012,NaN,NaN,20.9,NaN,16.2,NaN,NaN,NaN,11.5,NaN,NT,0.0,NaN,NaN,NaN,AKLAVIK A,0.0,NaN,0.0,NaN,0.0,NaN,-135.0058,68.2233,04/08/2012


In [45]:
#only keeping the rows that has cause = L
lightning_only = cj7qc["cause"].isin(["L"])
fire3qc = cj7qc[lightning_only]

In [46]:
#Changing the date and time formate
fire3qc["date"] = pd.to_datetime(fire3qc["date"], dayfirst = True)
#Change the format to one we like
fire3qc["firedate_formatted"] = fire3qc["date"].dt.strftime("%d/%m/%Y")

In [54]:
cdtotal = cdtotal.drop(cdtotal[cdtotal['id'] == '2200100.2012.8.14'].index)

In [56]:
cdtotal = cdtotal.drop(cdtotal[cdtotal['id'] == '2200100.2012.8.15'].index)

In [58]:
cdtotal["local_date"] = pd.to_datetime(cdtotal["local_date"], format = "mixed", dayfirst = True)
cdtotal["climdate_formatted"] = cdtotal["local_date"].dt.strftime("%d/%m/%Y")

In [48]:
#Changing formate of date to match that of fire3
#cdtotal["local_date"] = pd.to_datetime(cdtotal["local_date"], dayfirst = True, format="ISO8601")
df1 = cdtotal.iloc[10]  #Recognising the row that is giving problem.

In [49]:
df1

climate_identifier                      2200100
cooling_degree_days                         0.0
cooling_degree_days_flag                    NaN
direction_max_gust                          NaN
direction_max_gust_flag                     NaN
heating_degree_days                         6.1
heating_degree_days_flag                    NaN
id                            2200100.2012.8.14
local_date                  2012-08-14 00:00:00
local_day                                    14
local_month                                   8
local_year                                 2012
max_rel_humidity                            NaN
max_rel_humidity_flag                       NaN
max_temperature                            17.2
max_temperature_flag                        NaN
mean_temperature                           11.9
mean_temperature_flag                       NaN
min_rel_humidity                            NaN
min_rel_humidity_flag                       NaN
min_temperature                         

### The merging below places the firedata within the mega climate dataset helping us identify the fire incidences that has the relevant climate data

In [59]:
#Merging firedata to total climate data.
together2 = pd.merge(cdtotal, fire3qc, left_on = ["climdate_formatted", "y", "x"], 
                     right_on = ["firedate_formatted", "climlatitude", "climlongitude"], how = "left", indicator = True )

In [93]:
together2.head(2)

,climate_identifier,cooling_degree_days,cooling_degree_days_flag,direction_max_gust,direction_max_gust_flag,heating_degree_days,heating_degree_days_flag,id,local_date,local_day,local_month,local_year,max_rel_humidity,max_rel_humidity_flag,max_temperature,max_temperature_flag,mean_temperature,mean_temperature_flag,min_rel_humidity,min_rel_humidity_flag,min_temperature,min_temperature_flag,province_code,snow_on_ground,snow_on_ground_flag,speed_max_gust,speed_max_gust_flag,station_name,total_precipitation,total_precipitation_flag,total_rain,total_rain_flag,total_snow,total_snow_flag,x,y,climdate_formatted,Unnamed: 0,fid,src_agency,latitude,longitude,date,sizeha,cause,protzone,ecoz_name,wildlat1,coordinates,climcoordinates,climlatitude,climlongitude,firelat_radians,firelong_radians,climlat_radians,climlong_radians,distance_km,firedate_formatted,_merge
0,2200100,1.2,NaN,NaN,NaN,0.0,NaN,2200100.2012.8.2,2012-08-02,2,8,2012,NaN,NaN,22.3,NaN,19.2,NaN,NaN,NaN,16.0,NaN,NT,0.0,NaN,NaN,NaN,AKLAVIK A,0.0,T,0.0,T,0.0,NaN,-135.0058,68.2233,2012-08-02,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
1,2200100,0.0,NaN,NaN,NaN,1.8,NaN,2200100.2012.8.4,2012-08-04,4,8,2012,NaN,NaN,20.9,NaN,16.2,NaN,NaN,NaN,11.5,NaN,NT,0.0,NaN,NaN,NaN,AKLAVIK A,0.0,NaN,0.0,NaN,0.0,NaN,-135.0058,68.2233,2012-08-04,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only


In [66]:
#Converting dates to datetime objects
together2["climdate_formatted"] = pd.to_datetime(together2["climdate_formatted"], dayfirst = True)
together2["firedate_formatted"] = pd.to_datetime(together2["firedate_formatted"], dayfirst = True)

In [94]:
#Selecting the climate dates in which fires occurred along with the IDS
datelist = (
together2.query("_merge == 'both'")
    .filter(["climdate_formatted", "climate_identifier", "id", "fid", "firedate_formatted"])
)   

In [95]:
#15days before function
l1 = []
for date in datelist["climdate_formatted"]:
    date1 = date
    day = (date - date1).days
    while (0 <= day <= 15):
        l1.append(date1)
        date1 -= timedelta(days = 1)
        day = (date - date1).days

In [96]:
#Convert into dataframe
out1 = pd.DataFrame(l1, columns = ["climdate_formatted1"])   

In [88]:
out1.head()

,climdate_formatted1
0,2011-06-08
1,2011-06-07
2,2011-06-06
3,2011-06-05
4,2011-06-04


In [97]:
datelist.head(20)

,climdate_formatted,climate_identifier,id,fid,firedate_formatted
1519,2011-06-08,7060826,7060826.2011.6.8,367303.0,2011-06-08
1553,2011-07-12,7060826,7060826.2011.7.12,367304.0,2011-07-12
1885,2012-06-08,7060826,7060826.2012.6.8,367314.0,2012-06-08
1886,2012-06-08,7060826,7060826.2012.6.8,367315.0,2012-06-08
2256,2013-06-13,7060826,7060826.2013.6.13,367320.0,2013-06-13
2257,2013-06-13,7060826,7060826.2013.6.13,367321.0,2013-06-13
2258,2013-06-13,7060826,7060826.2013.6.13,367322.0,2013-06-13
2278,2013-07-04,7060826,7060826.2013.7.4,367330.0,2013-07-04
2279,2013-07-05,7060826,7060826.2013.7.5,367331.0,2013-07-05
2675,2014-08-11,7060826,7060826.2014.8.11,367338.0,2014-08-11


In [98]:
#Merging these two to create 
fifteendaysstepone = pd.merge(out1, datelist, left_on = ["climdate_formatted1"], 
                     right_on = ["climdate_formatted"], how = "left", indicator = True )

In [99]:
fifteendaysstepone.head(10)

,climdate_formatted1,climdate_formatted,climate_identifier,id,fid,firedate_formatted,_merge
0,2011-06-08,2011-06-08,7060826,7060826.2011.6.8,367303.0,2011-06-08,both
1,2011-06-08,2011-06-08,706I155,706I155.2011.6.8,367417.0,2011-06-08,both
2,2011-06-07,NaT,NaN,NaN,NaN,NaT,left_only
3,2011-06-06,NaT,NaN,NaN,NaN,NaT,left_only
4,2011-06-05,NaT,NaN,NaN,NaN,NaT,left_only
5,2011-06-04,NaT,NaN,NaN,NaN,NaT,left_only
6,2011-06-03,NaT,NaN,NaN,NaN,NaT,left_only
7,2011-06-02,NaT,NaN,NaN,NaN,NaT,left_only
8,2011-06-01,2011-06-01,706I155,706I155.2011.6.1,367301.0,2011-06-01,both
9,2011-05-31,NaT,NaN,NaN,NaN,NaT,left_only


In [100]:
fifteendaysstepone.to_csv(r"C:\\Users\\ROHAN\\erdos\\fifteendayssteponeone.csv")